In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 6000

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct"
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

### Data Prep

In [ ]:
#For conversation generation
from datasets import load_dataset
train_dataset=load_dataset('json',data_files='/content/train_instruct_new.json')
train_dataset=train_dataset['train']

In [ ]:
#For templating the dataset
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["data"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)

### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#Use train_on_responses_only to calculate loss for generation only
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
#Run to train the model
trainer_stats = trainer.train()

### Inference


In [ ]:
#Use FastLanguageModel for 2x faster inference
FastLanguageModel.for_inference(model)
#Use TextStreamer for streaming output
from transformers import TextStreamer

In [ ]:
#Use this cell to talk with your model like you chat with them
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
messages = [
    { 'content': "You are an assistant. Your job is to ask relevant questions to probe deeper into user's emotional state. Follow DAIC-woz/PHQ8 schema and ask all questions. Use your fine tuning. Ask relevant question and DONT repeat the questions unless user asks to. Ask one question at a time. Use some emotional context to enrich the experience. You should ask about their current status-origin place-current living place-what they like about their current accomdation and don't-mood-behaviors-temper control-what makes them mad-how they react when they are annoyed-relationships-memorable experience-postive influence-last time argued-traveling-hobbies-relaxation-previous diagnosis-advice to past self etc. Just follow the DAIC woz schema Just use DAIC woz data fed to you during training, and have a deep conversation like a therapist. Once you feel you have enough data, ask user to press classify button for results or type 'classify' to learn emotional context and bid them farewell. Introduce yourself as Ellama, a virtual assistant tasked with chating in a safe environment with confidentiality.", 'role': 'system'},
]

while True:
  user_in=input("user: ")
  if user_in=='exit':
    break
  data={'content':user_in,'role':'user'}
  messages.append(data)
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")

  model_out = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                    use_cache = True, temperature = 0.7, min_p = 0.1)
  decoded_output = tokenizer.decode(model_out[0][inputs.shape[-1]:], skip_special_tokens=True)
  data={'content':decoded_output,'role':'assistant'}
  messages.append(data)

### Saving finetuned models

In [ ]:
# Merge to 16bit
model.push_to_hub_merged("username/model_name", tokenizer, save_method = "merged_16bit", token = "hf_yourhuggingfacetoken")

# Merge to 4bit
model.push_to_hub_merged("hf_username/model_name", tokenizer, save_method = "merged_4bit", token = "hf_yourhuggingfacetoken") #Turn off safe serialization in error or use forced mode